## Setup

In [ ]:
!pip install poetry

In [ ]:
!pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [1]:
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')
if not on_colab:
    import napari
import smorph.util.autocrop as ac

In [2]:
def view_3D(*args):  # Models a 3D image
    n_images = len(args)
    if not on_colab and n_images > 0:
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3)
            for itr in range(1, n_images):
                del args[itr]['colormap']
                del args[itr]['gamma']
                viewer.add_labels(**args[itr])

## Select somas of individual cells in clumps

In [3]:
import numpy as np
SOMA_SELECTED = 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue'
reconstructed_labels = None  # np.zeros(original.shape)
reconstructed_labels, parent_path, roi_path = ac.postprocess_segment(SOMA_SELECTED, reconstructed_labels)

[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue/1213ab3986434302a9b03523781766db_mip.roi'
[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue/1d89eb85299846b28f4e58981478e633_mip.roi'
[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue/36323b4773d041c2b07108d117d920ba_mip.roi'
[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue/3f63117220074649bb23b38222b3ae49_mip.roi'
[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2.1MA_3_LONG MARK_20X_SEC 1_LEFT HILUS_21 DAY-ML/residue/470698544c524d6098d43be00bb723b5_mip.roi'
[Errno 2] No such file or directory: 'Autocropped/CTRLvDMI-21-ML-Deconv/test/DMI_MSP2

In [4]:
view_3D(#{'data': labels, 'colormap': 'gray', 'name': 'labels'},
        {'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

In [7]:
reconstructed_volume = (reconstructed_labels > 0).sum()
f'Postreconstructing Volume: {reconstructed_volume}'

'Postreconstructing Volume: 139300'

In [8]:
linebuilder = ac._roi_extract._load_ROI(roi_path)

In [13]:
import numpy as np
from skimage.morphology import binary_erosion
from skimage.segmentation import clear_border
from skimage.draw import polygon2mask

conservative = True
reconstructed_residue_labels = clear_border(reconstructed_labels, mask=None if conservative
                        else ac.core._compute_convex_hull(thresholded))

if linebuilder is not None:
    X, Y = ac.core._unwrap_polygon(linebuilder)
    min_x, max_x = int(min(X)), int(max(X) + 1)
    min_y, max_y = int(min(Y)), int(max(Y) + 1)
    shape = reconstructed_labels.shape
    roi_mask = np.empty(shape)
    roi_mask[0] = polygon2mask(shape[1:][::-1], list(zip(X, Y))).T

    for i in range(1, shape[0]):
        roi_mask[i] = roi_mask[0]
    roi_mask = binary_erosion(roi_mask)
    reconstructed_residue_labels = clear_border(reconstructed_residue_labels, mask=roi_mask)

reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

In [14]:
view_3D({'data': reconstructed_residue_labels, 'colormap': 'gray', 'name': 'reconstructed_filtered_regions'},
        {'data': reconstructed_labels, 'gamma': .8, 'colormap': 'inferno', 'name': 'reconstructed_labels'},
        {'data': roi_mask, 'gamma': .8, 'colormap': 'inferno', 'name': 'roi_mask'})

In [20]:
reconstructed_residue_labels = ac.filter_labels(reconstructed_labels, None, linebuilder, True)

In [21]:
reconstructed_residue_volume = (reconstructed_residue_labels > 0).sum()
f'Postreconstructing-residue Volume: {reconstructed_residue_volume}; ' \
f'%age volume: {(reconstructed_residue_volume) / prefiltering_volume * 100} %'

'Postreconstructing-residue Volume: 48795; %age volume: 21.12913911586277 %'

In [24]:
view_3D({'data': denoised, 'colormap': 'inferno', 'name': 'denoised'},
        {'data': reconstructed_residue_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'reconstructed_filtered_labels'},
        {'data': filtered_labels, 'colormap': 'inferno', 'gamma': .8, 'name': 'filtered_labels'},
        {'data': labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

In [26]:
reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

LOW_VOLUME_CUTOFF = 0  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = 1e9  # filter cell clusters

OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'segmented'

ac.export_cells(parent_path, LOW_VOLUME_CUTOFF,
                HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, denoised,
                reconstructed_filtered_regions, None, SEGMENT_TYPE, NAME_ROI, linebuilder, roi_path=roi_path)